# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [12]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15, 20, 25]},
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 1h/4h, trade type is sell
Number of combinations is 12


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:44<00:00,  8.69s/it]


# Check local statistics

In [2]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
0,STOCH_RSI,14,20,9,7,3,15,0.8078,0.6100,0.5667,0.5557,0.4657,0.5575,0.5690,0.7255,0.9185,0.9540,1.1524,1.4610,1.5012,1.5999,1.7793,1.8013,1.7749,1.7749,1.7749,1.7749,1.7749,1.7749,1.7749,1.7749,-0.16,-0.02,-0.14,-0.00,0.00,0.00,-0.03,-0.20,-0.46,-0.50,-0.98,-1.39,-1.51,-1.34,-1.53,-1.53,-1.54,-1.46,-1.69,-1.65,-1.53,-1.53,-1.62,-1.72,10,1.259362,-0.938750,2.593625,-9.387500
1,STOCH_RSI,14,20,9,7,3,20,0.7483,0.5607,0.5190,0.5034,0.5080,0.5726,0.6622,0.7601,0.8640,0.9076,0.9951,1.1312,1.1434,1.1838,1.2428,1.2535,1.2799,1.3396,1.4491,1.5294,1.5647,1.5740,1.5740,1.5740,-0.22,0.00,-0.06,-0.00,0.00,-0.09,-0.35,-0.40,-0.46,-0.51,-0.75,-0.78,-0.70,-0.78,-0.43,-0.52,-0.71,-1.36,-1.39,-1.52,-1.40,-1.53,-1.52,-1.49,22,1.060017,-0.707083,1.320367,-15.555833
2,STOCH_RSI,14,20,9,7,3,25,0.7483,0.5607,0.5190,0.5034,0.5080,0.5726,0.6622,0.7601,0.8640,0.9076,0.9951,1.1312,1.1434,1.1838,1.2428,1.2535,1.2799,1.3396,1.4491,1.5294,1.5647,1.5740,1.5740,1.5740,-0.22,0.00,-0.06,-0.00,0.00,-0.09,-0.35,-0.40,-0.46,-0.51,-0.75,-0.78,-0.70,-0.78,-0.43,-0.52,-0.71,-1.36,-1.39,-1.52,-1.40,-1.53,-1.52,-1.49,22,1.060017,-0.707083,1.320367,-15.555833
3,STOCH_RSI,14,25,9,7,3,15,0.9718,0.6989,0.6992,0.7263,0.6306,0.6622,0.6451,0.6724,0.7148,0.7087,0.7457,0.8558,0.8828,0.8833,0.9697,0.9587,0.9768,0.9516,0.9782,1.0133,0.9705,0.9730,0.9404,0.9586,-0.19,-0.03,0.00,-0.31,-0.19,-0.17,-0.07,0.00,-0.21,-0.20,-0.14,-0.17,-0.29,-0.29,-0.33,-0.29,-0.47,-0.79,-0.73,-0.70,-0.65,-0.78,-0.65,-1.04,21,0.841183,-0.362083,-3.335150,-7.603750
4,STOCH_RSI,14,25,9,7,3,20,0.9326,0.7718,0.7421,0.7496,0.7311,0.7550,0.7743,0.8276,0.8378,0.8546,0.8469,0.8823,0.8898,0.8962,0.9413,0.9345,0.9612,0.9702,1.0479,1.1558,1.1521,1.1610,1.1537,1.1700,-0.26,-0.17,-0.15,-0.39,-0.33,-0.23,-0.29,-0.40,-0.38,-0.31,-0.19,-0.13,-0.19,-0.13,-0.17,-0.29,-0.47,-0.68,-1.22,-1.16,-1.00,-1.01,-0.99,-1.16,65,0.922475,-0.487500,-5.039125,-31.687500
5,STOCH_RSI,14,25,9,7,3,25,0.8882,0.7393,0.6768,0.6927,0.6819,0.7044,0.7418,0.7848,0.8102,0.8229,0.8183,0.8443,0.8501,0.8561,0.8938,0.8823,0.9051,0.9208,0.9907,1.0858,1.0865,1.0944,1.0893,1.1088,-0.28,-0.18,-0.26,-0.41,-0.32,-0.22,-0.30,-0.41,-0.46,-0.35,-0.22,-0.15,-0.19,-0.10,-0.13,-0.28,-0.45,-0.64,-1.09,-1.16,-1.00,-1.00,-0.98,-1.16,78,0.873721,-0.489167,-9.849775,-38.155000
6,STOCH_RSI,14,30,9,7,3,15,0.7723,0.7365,0.7823,0.8728,0.8568,0.8907,0.8301,0.6989,0.6980,0.6576,0.6329,0.6757,0.6355,0.6083,0.6422,0.6508,0.6521,0.6467,0.6488,0.6628,0.6595,0.6631,0.6634,0.6744,-0.24,-0.14,-0.29,-0.31,-0.33,-0.23,-0.07,0.00,-0.01,-0.16,-0.16,-0.04,0.00,-0.08,-0.30,-0.33,-0.45,-0.33,-0.40,-0.36,-0.42,-0.34,-0.41,-0.57,54,0.704675,-0.248750,-15.947550,-13.432500
7,STOCH_RSI,14,30,9,7,3,20,0.7873,0.7450,0.7431,0.7835,0.8122,0.8073,0.7940,0.7614,0.7680,0.7559,0.7354,0.7531,0.7372,0.7220,0.7578,0.7693,0.7952,0.8053,0.8407,0.8951,0.9030,0.9063,0.9080,0.9105,-0.23,-0.18,-0.24,-0.32,-0.35,-0.25,-0.18,-0.14,-0.06,-0.19,-0.13,-0.03,0.01,0.00,-0.08,-0.32,-0.48,-0.61,-0.72,-0.86,-0.77,-0.77,-0.82,-0.80,134,0.799858,-0.355000,-26.818983,-47.570000
8,STOCH_RSI,14,30,9,7,3,25,0.8349,0.7891,0.7697,0.7992,0.8370,0.8277,0.8283,0.8086,0.8222,0.8042,0.7

In [13]:
# 1h/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
10,STOCH_RSI,14,35,9,7,3,20,0.9234,0.7884,1.1282,1.2423,1.5703,1.7443,1.7507,2.3035,2.2814,2.0919,2.0435,1.4431,1.2322,1.0719,0.9922,0.9698,0.9480,0.9406,0.9037,0.8589,0.8439,0.8104,0.7647,0.7573,-0.26,-0.14,-0.94,-0.84,-1.45,-1.28,-1.37,-2.52,-2.31,-1.62,-1.20,-0.58,0.51,0.90,1.60,1.61,1.48,1.36,1.28,0.98,1.08,1.61,2.23,2.32,55,1.266858,0.102083,14.677208,5.614583
11,STOCH_RSI,14,35,9,7,3,25,0.9429,0.8433,1.1398,1.2081,1.4993,1.6383,1.6279,2.0925,2.0757,1.9200,1.8634,1.3878,1.2057,1.0650,0.9946,0.9732,0.9515,0.9382,0.9015,0.8580,0.8408,0.8183,0.7763,0.7696,-0.33,-0.39,-0.95,-0.90,-1.45,-1.28,-1.38,-2.38,-2.29,-1.61,-1.20,-0.58,0.38,0.84,1.25,1.19,1.02,1.16,0.97,0.98,1.07,1.61,2.07,2.28,63,1.222154,0.003333,13.995712,0.210000
4,STOCH_RSI,14,25,9,7,3,20,1.3349,1.7705,2.2824,2.6408,2.9190,3.7010,3.3872,3.6336,2.9123,3.0156,3.0156,1.5896,1.4785,1.3702,1.3450,1.3205,1.2395,1.2395,1.2395,0.9672,0.9672,0.9672,0.9672,0.9672,-0.27,-0.51,-0.77,-0.48,-1.66,-1.15,-1.79,-2.38,-2.43,-1.09,-0.85,-0.83,-0.53,-0.32,-0.21,-0.16,-0.06,-0.21,-0.21,-0.21,0.05,0.00,-0.16,-0.11,7,1.927967,-0.680833,6.495767,-4.765833
5,STOCH_RSI,14,25,9,7,3,25,1.3349,1.7705,2.2824,2.6408,2.9190,3.7010,3.3872,3.6336,2.9123,3.0156,3.0156,1.5896,1.4785,1.3702,1.3450,1.3205,1.2395,1.2395,1.2395,0.9672,0.9672,0.9672,0.9672,0.9672,-0.27,-0.51,-0.77,-0.48,-1.66,-1.15,-1.79,-2.38,-2.43,-1.09,-0.85,-0.83,-0.53,-0.32,-0.21,-0.16,-0.06,-0.21,-0.21,-0.21,0.05,0.00,-0.16,-0.11,7,1.927967,-0.680833,6.495767,-4.765833
7,STOCH_RSI,14,30,9,7,3,20,0.5653,0.6033,1.0197,1.1564,1.5346,1.7683,1.7627,2.4778,2.1792,2.2156,2.2156,1.4911,1.3893,1.2343,1.1653,1.1393,1.0711,1.0711,1.0543,0.9456,0.9456,0.8797,0.8640,0.8640,-0.12,0.04,-1.05,-0.90,-1.70,-1.15,-1.39,-3.28,-2.43,-1.39,-1.05,-0.86,0.14,0.39,1.02,0.90,0.96,1.10,0.56,0.42,0.56,1.15,1.60,1.10,17,1.317217,-0.224167,5.392683,-3.810833
9,STOCH_RSI,14,35,9,7,3,15,0.7698,0.6201,0.9801,1.0888,1.4688,1.6124,1.5975,2.0642,1.9864,1.7674,1.7290,1.2196,1.0590,0.9247,0.8599,0.8389,0.8327,0.8279,0.8021,0.7614,0.7605,0.7372,0.7088,0.7051,-0.14,-0.05,-0.83,-0.77,-1.45,-1.21,-1.26,-2.66,-2.29,-1.53,-1.05,-0.19,0.80,1.58,2.09,2.16,1.74,2.10,1.78,1.59,1.82,2.11,3.04,2.45,35,1.113429,0.409583,3.970021,14.335417
3,STOCH_RSI,14,25,9,7,3,15,2.6523,3.1877,3.9418,4.9086,3.8176,5.4194,3.9417,3.2413,3.6119,3.6119,3.6119,1.3987,1.2220,1.1331,1.1030,1.0714,1.0045,1.0045,1.0045,0.6701,0.6701,0.6701,0.6701,0.6701,-0.27,-0.27,-0.32,-0.48,-0.53,-0.85,-0.69,-1.08,-0.90,-0.64,-0.37,-0.21,-0.53,-0.32,-0.21,-0.16,-0.16,-0.21,-0.21,-0.21,0.05,0.00,-0.16,-0.11,3,2.259929,-0.368333,3.779787,-1.105000
8,STOCH_RSI,14,30,9,7,3,25,0.5613,0.6008,0.9189,0.9802,1.3056,1.3850,1.3599,1.8869,1.7536,1.7800,1.8039,1.3482,1.2850,1.1669,1.1126,1.0919,1.0367,1.0367,1.0230,0.9324,0.9324,0.8969,0.8830,0.8830,-0.22,-0.33,-0.93,-0.84,-1.68,-1.08,-1.56,-3.20,-2.55,-1.62,-1.14,-0.87,-0.08,0.34,0.73,0.39,0.41,0.61,0.40,0.40,0.47,0.96,1.52,1.06,20,1.165200,-0.367083,3.304000,-7.341667
6,STOCH_RSI,14,30,9,7,3,15,0.6037,0.5829,0.9860,1.1628,1.4583,1.7300,1.6475,2.0287,1.9923,1.9923,1.9923,1.3733,1.2657,1.1546,1.1038,1.0679,1.0433,1.0433,1.0178,0.8630,0.863

In [3]:
# 4h/1d
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
58,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,0.8869,0.7709,0.6672,0.7839,1.0367,1.0548,1.0683,1.0804,1.1496,1.1957,1.1847,1.1210,1.0655,1.0655,1.0655,1.2117,1.6926,1.6926,1.8096,1.8421,1.9108,2.4737,2.4737,2.4737,-0.67,-0.47,-0.48,-0.92,-1.68,-1.10,-0.74,-0.85,-0.77,-0.58,0.18,0.00,-0.31,-0.47,-0.73,-1.37,-1.95,-1.54,-1.58,-2.11,-2.35,-4.55,-4.35,-3.93,6,1.365712,-1.388333,2.194275,-8.330000
53,STOCH_RSI_LinearReg,14,35,9,7,3,20,10,0,0.8869,0.7709,0.6672,0.7839,1.0367,1.0548,1.0683,1.0804,1.1496,1.1957,1.1847,1.1210,1.0655,1.0655,1.0655,1.2117,1.6926,1.6926,1.8096,1.8421,1.9108,2.4737,2.4737,2.4737,-0.67,-0.47,-0.48,-0.92,-1.68,-1.10,-0.74,-0.85,-0.77,-0.58,0.18,0.00,-0.31,-0.47,-0.73,-1.37,-1.95,-1.54,-1.58,-2.11,-2.35,-4.55,-4.35,-3.93,6,1.365712,-1.388333,2.194275,-8.330000
59,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,1.7290,1.5608,1.1393,1.0245,1.2420,1.2920,1.2943,1.2956,1.3560,1.3963,1.3963,1.3034,1.3034,1.3197,1.1043,1.0469,1.3986,1.2435,1.2393,1.1898,1.1588,1.1719,1.1719,1.1719,-0.92,-0.97,-1.80,-0.99,-1.63,-1.77,-1.03,-0.71,-0.96,-0.80,-0.13,-0.13,-0.42,-0.74,-0.61,-1.77,-2.18,-2.12,-1.77,-2.18,-2.44,-2.44,-1.22,-0.88,7,1.272896,-1.275417,1.910271,-8.927917
54,STOCH_RSI_LinearReg,14,35,9,7,3,20,12,0,1.7290,1.5608,1.1393,1.0245,1.2420,1.2920,1.2943,1.2956,1.3560,1.3963,1.3963,1.3034,1.3034,1.3197,1.1043,1.0469,1.3986,1.2435,1.2393,1.1898,1.1588,1.1719,1.1719,1.1719,-0.92,-0.97,-1.80,-0.99,-1.63,-1.77,-1.03,-0.71,-0.96,-0.80,-0.13,-0.13,-0.42,-0.74,-0.61,-1.77,-2.18,-2.12,-1.77,-2.18,-2.44,-2.44,-1.22,-0.88,7,1.272896,-1.275417,1.910271,-8.927917
44,STOCH_RSI_LinearReg,14,30,9,7,3,25,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
39,STOCH_RSI_LinearReg,14,30,9,7,3,20,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
34,STOCH_RSI_LinearReg,14,30,9,7,3,15,12,0,4.0536,4.0536,1.1195,1.1195,1.2121,1.2276,1.2739,1.2739,1.2739,1.2739,1.2739,0.9944,0.9944,1.0482,1.0482,1.0482,2.0675,2.0675,2.0675,2.0675,2.1639,2.2483,2.2483,2.2483,-0.78,-0.74,-1.33,-1.21,-1.26,-1.35,-1.29,-0.91,-1.26,-0.76,-0.03,-0.28,-0.71,-1.44,-1.14,-1.48,-3.10,-2.92,-2.03,-2.50,-2.71,-2.55,-1.46,-1.18,2,1.727817,-1.434167,1.455633,-2.868333
52,STOCH_RSI_LinearReg,14,35,9,7,3,20,8,0,1.0293,1.0050,1.0050,1.0227,1.9329,2.0776,2.0776,2.1133,2.1133,1.7363,1.1648,1.0662,1.0662,1.0662,1.0662,1.0662,1.0662,1.0662,1.2601,1.2051,1.1844,1.6578,1.6578,1.6578,-0.71,-1.18,-0.79,-0.99,-2.97,-2.55,-2.90,-2.52,-2.10,-1.09,0.08,-0.67,-0.42,-1.01,-1.51,-1.43,-0.67,0.42,0.50,0.85,-0.28,-3.69,-3

In [2]:
# 4h/1d no LR
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
5,STOCH_RSI,14,25,9,7,3,25,0.8545,1.5017,1.5017,1.5017,2.7068,3.1011,3.3503,3.3503,3.3503,5.6591,5.8437,6.8674,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,8.1952,-0.74,-1.30,-0.63,-1.03,-2.35,-2.69,-2.91,-2.52,-2.36,-4.91,-5.07,-5.96,-7.11,-6.82,-6.40,-6.36,-6.47,-6.44,-6.23,-6.13,-6.04,-6.35,-6.29,-6.28,1,5.747125,-4.557917,4.747125,-4.557917
7,STOCH_RSI,14,30,9,7,3,20,0.0833,0.0500,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,1.9583,2.7500,3.1667,3.3333,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,1.6154,-0.07,0.27,0.80,1.07,1.07,0.93,0.00,0.00,-0.07,-0.07,-0.07,-3.13,-4.40,-5.06,-5.33,-5.60,-5.60,-4.53,-4.26,-4.20,-2.66,-2.40,-3.20,-3.20,1,1.722179,-2.071250,0.722179,-2.071250
8,STOCH_RSI,14,30,9,7,3,25,0.0833,0.0500,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,1.9583,2.7500,3.1667,3.3333,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,1.6154,-0.07,0.27,0.80,1.07,1.07,0.93,0.00,0.00,-0.07,-0.07,-0.07,-3.13,-4.40,-5.06,-5.33,-5.60,-5.60,-4.53,-4.26,-4.20,-2.66,-2.40,-3.20,-3.20,1,1.722179,-2.071250,0.722179,-2.071250
10,STOCH_RSI,14,35,9,7,3,20,0.0833,0.0500,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,1.9583,2.7500,3.1667,3.3333,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,1.6154,-0.07,0.27,0.80,1.07,1.07,0.93,0.00,0.00,-0.07,-0.07,-0.07,-3.13,-4.40,-5.06,-5.33,-5.60,-5.60,-4.53,-4.26,-4.20,-2.66,-2.40,-3.20,-3.20,1,1.722179,-2.071250,0.722179,-2.071250
11,STOCH_RSI,14,35,9,7,3,25,0.0833,0.0500,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,0.0417,1.9583,2.7500,3.1667,3.3333,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,3.5000,1.6154,-0.07,0.27,0.80,1.07,1.07,0.93,0.00,0.00,-0.07,-0.07,-0.07,-3.13,-4.40,-5.06,-5.33,-5.60,-5.60,-4.53,-4.26,-4.20,-2.66,-2.40,-3.20,-3.20,1,1.722179,-2.071250,0.722179,-2.071250
0,STOCH_RSI,14,20,9,7,3,15,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.000000,0.000000,-0.000000,0.000000
6,STOCH_RSI,14,30,9,7,3,15,0.2000,0.2000,0.8000,0.8000,0.3556,0.5111,0.4211,0.4211,0.4211,0.4211,0.4211,0.4211,0.4211,0.4211,0.4211,0.4211,0.7018,0.8772,0.8947,0.8361,0.8361,0.8361,0.6986,0.6296,-0.29,-0.07,-1.15,-0.64,-1.15,-1.65,-1.72,-1.36,-1.00,0.29,1.07,-0.21,-0.14,-0.07,-0.57,-0.93,-2.87,-3.58,-3.65,-3.58,1.15,1.07,1.58,1.79,1,0.557829,-0.736667,-0.442171,-0.736667
3,STOCH_RSI,14,25,9,7,3,15,0.3929,0.3143,0.1833,0.1833,0.1833,0.1549,0.1089,0.1089,0.1089,0.1089,0.1089,0.1089,0.1089,0.1089,0.1089,0.1089,0.2772,0.3069,0.3069,0.3069,0.3069,0.3069,0.3069,0.3069,-1.05,0.19,-0.86,-0.29,-0.48,1.05,3.74,5.27,2.78,3.26,1.44,1.63,0.29,-0.67,-0.86,-0.29,-2.68,-2.97,-1.53,0.29,1.53,2.11,2.88,-0.38,1,0.205271,0.600000,-0.794729,0.600000
9,STOCH_RSI,14,35,9,7,3,15,0.7096,0.5248,0.8650,0.4192,0.3091,0.5215,0.3672,0.4776,0.7106,0.8220,0.8220,0.8220,0.8220,0.8220,0.7516,0.7516,0.8414,0.8975,0.9031,0.8833,0.8833,0.8833

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '4h'
higher_timeframe = '1d'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [20]},
                'LinearReg': {'timeperiod': [12], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            